In [30]:

from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, FinalAnswerTool, Tool, tool 

# Create a model using LiteLLMModel with Ollama
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5-coder:32b",  # Format: "ollama_chat/[model-name]"
    api_base="http://localhost:11434",   # Default Ollama API endpoint
    api_key="ollama",                    # This is just a placeholder, Ollama doesn't actually require an API key
    num_ctx=30000                        # Ollama default is 2048 which might be too small for complex tasks
)

# Create the CodeAgent with all tools
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        FinalAnswerTool()
    ],
    model=model,
    additional_authorized_imports=["wikipedia", "requests", "json", "re", "datetime", "os"]
)

In [31]:
result = agent.run("If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it │
│ take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum        │
│ perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the   │
│ nearest 1000 hours and do not use any comma separators if necessary.                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:32b ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  kipchoge_pace = web_search("Eliud Kipchoge marathon record")                                                     
  moon_perigee = web_search("Moon closest approach to Earth")                                                      
  print(f"Kipchoge's marathon pace: {kipchoge_pace}")                                                              
  print(f"Moon perigee distance: {moon_perigee}")                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'kipchoge_pace = web_search("Eliud Kipchoge marathon record")' due to: 
DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

[Step 1: Duration 66.60 seconds| Input tokens: 2,184 | Output tokens: 124]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extracting relevant data                                                                                       
  record_time_str = "2:01:39"  # format HH:MM:SS                                                                   
                                                                                                                   
  # Splitting the time string into hours, minutes, and seconds                                                     
  hours, minutes, seconds = map(int, record_time_str.split(':'))                                                   
                                                                                                                   
  # Converting the marathon time to total seconds                                                                  
  total_seconds = hours * 3600 + minutes * 60 + seconds                                                            
                                                                                                                   
  # Marathon distance in kilometers                                                                                
  marathon_distance_km = 42.195                                                                                    
                                                                                                                   
  # Kipchoge's speed in km/s                                                                                       
  speed_kms = marathon_distance_km / total_seconds                                                                 
                                                                                                                   
  # Moon perigee distance in kilometers                                                                            
  moon_perigee_distance_km = 363104                                                                                
                                                                                                                   
  # Time to run the distance between Earth and the Moon in seconds                                                 
  time_to_moon_seconds = moon_perigee_distance_km / speed_kms                                                      
                                                                                                                   
  # Converting time to hours                                                                                       
  time_to_moon_hours = time_to_moon_seconds / 3600                                                                 
                                                                                                                   
  # Rounding to the nearest thousand hours                                                                         
  time_to_moon_rounded = round(time_to_moon_hours, -3)                                                             
                                                                                                                   
  print(f"Time to run from Earth to Moon: {time_to_moon_rounded} hours")                                           
  final_answer(int(time_to_moon_rounded))                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Time to run from Earth to Moon: 17000.0 hours

Out - Final answer: 17000

[Step 3: Duration 14.55 seconds| Input tokens: 10,064 | Output tokens: 619]

17000
